In [1]:
import ipyparallel as ipp
c = ipp.Client()
dview = c.direct_view()

# Ejercicios

Determinar si un numero de la forma $2^n -1$, donde $n$ es un entero, es primo o no. Los primos que tienen esta forma se llaman primos de Mersenne.

1.- Escribir una funcion que determine si el numero dado por $2^n -1$ es primo (usar la funcion anterior)

In [2]:
# Función que devuelve una lista con los numeros primos del 2 a n
def prime(n):
    nP = []
    for i in xrange(2,n):
        for j in xrange(2,i):
            if (i % j == 0):
                break
        else:
            nP.append(2**i-1)
    return nP

In [3]:
# Función que devuelve x si es un numero primo de Mersenne
PrimeM = lambda x: x if (all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1) else None

2.- Seleccionar los primos de Mersenne de entre los primeros 5000 numeros enteros (nota: el resultado es {2, 3, 5, 7, 13, 17, 19, 31, 61, 89, 107, 127, 521, 607, 1279, 2203, 2281, 3217, 4253, 4423}

In [14]:
# Código que devuelve una lista con los numeros primos de Mersenne usando las dos funciones anteriores
filter(bool,list((map(PrimeM, prime(15)))))

[3, 7, 31, 127, 8191]

3.- Estimar el tiempo que su computadora tarda en encontrar los primero 5000 primos de Mersenne

In [15]:
%timeit filter(bool,list((map(PrimeM, prime(15)))))

The slowest run took 6.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 30.9 µs per loop


In [4]:
%timeit filter(bool,list((dview.map_sync(PrimeM, prime(15)))))

The slowest run took 48.98 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 26.2 ms per loop


# Tarea

1.- Estudiar la informacion contenida en el archivo /.ipython/profile_default/security/ipcontroller-client.json y explicar en un parrafo que significa. Leer la documentacion

In [ ]:
# %load ~/.ipython/profile_default/security/ipcontroller-client.json
{
  "control": 62163, 
  "task": 62169, 
  "notification": 62173, 
  "task_scheme": "leastload", 
  "mux": 62165, 
  "iopub": 62171, 
  "ssh": "", 
  "key": "22bddeae-97f7-4a8d-bfe5-91f422425951", 
  "registration": 62161, 
  "interface": "tcp://127.0.0.1", 
  "signature_scheme": "hmac-sha256", 
  "pack": "json", 
  "unpack": "json", 
  "location": "192.168.10.181"
}

# ipcontroller-client.json

El fichero ipcontroller-client.json contiene información relacionada al cliente para que este pueda conectarse y autenticarse al controlador el cual es el proceso que administra la comunicación y el paso de mensajes y datos

2.- Investigar el uso de los metodos apply,apply_sync de las instancias DirectView (el objeto dview) y discutir las diferencias con map y map_sync, por medio de ejemplos. (Ver la documentacion)

La función <span style="color: green">_map_</span> permite aplicar una funcion dada a cada elemento de un iterable (lista,tupla, etc.) de manera **secuencial** y devolver una lista de los resultados.

In [6]:
%timeit serial_result = map(lambda x:x**10, range(32))

100000 loops, best of 3: 7.13 µs per loop


La función <span style="color: green">_map__sync_</span> utiliza los motores para aplicar de forma **paralelizada** una funcion dada a cada elemento de un iterable (lista,tupla, etc.) y devolver una lista de los resultados.

In [8]:
%timeit parallel_result = dview.map_sync(lambda x: x**10, range(32))

10 loops, best of 3: 40.9 ms per loop


La función <span style="color: green">_apply_</span> permite ejecutar una funcion de forma paralelizada (sin bloqueo) usando los motores del objeto DirectView, en caso de que los motores asignados esten ocupado regresa un objeto _AsyncResult_ el cual puede ser concultado porteriormente mediante get()

In [12]:
AS = dview.apply(lambda: "hola clase")

In [13]:
AS.get()

['hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase']

La función <span style="color: green">_apply__sync_</span> permite ejecutar una funcion de forma paralelizada (con bloqueo) usando los motores del objeto DirectView

In [16]:
dview.apply_sync(lambda: "hola clase")

['hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase',
 'hola clase']

3.- Determinar que partes de un programa que encuentra los primeros 5000 primos de Mersenne son paralelizables y sugerir como podrian paralelizarlo

4.- Aplicar el metodo apply y/o map descritos aqui al programa que selecciona los primos de Mersenne de entre los primeros 5000 numeros enteros